In [1]:
import geopandas as gpd
import pandas as pd
from pytz import timezone
import folium
from shapely.geometry import Polygon
import os

# Load rainfall records CSV
rainfall_data = pd.read_csv('rainfall_records.csv')

# VTEC dictionaries (already provided by you)
VTEC_PHENOMENA = {
    "AF": "Ashfall", "AS": "Air Stagnation", "BH": "Beach Hazard", "BS": "Blowing Snow",
    "BW": "Brisk Wind", "BZ": "Blizzard", "CF": "Coastal Flood", "CW": "Cold Weather",
    "DF": "Debris Flow", "DS": "Dust Storm", "DU": "Blowing Dust", "EC": "Extreme Cold",
    "EH": "Excessive Heat", "EW": "Extreme Wind", "FA": "Flood", "FF": "Flash Flood",
    "FG": "Dense Fog", "FL": "Flood", "FR": "Frost", "FW": "Red Flag", "FZ": "Freeze",
    "UP": "Freezing Spray", "GL": "Gale", "HF": "Hurricane Force Wind", "HI": "Inland Hurricane",
    "HS": "Heavy Snow", "HT": "Heat", "HU": "Hurricane", "HW": "High Wind", "HY": "Hydrologic",
    "HZ": "Hard Freeze", "IP": "Sleet", "IS": "Ice Storm", "LB": "Lake Effect Snow and Blowing Snow",
    "LE": "Lake Effect Snow", "LO": "Low Water", "LS": "Lakeshore Flood", "LW": "Lake Wind",
    "MA": "Marine", "MF": "Marine Dense Fog", "MH": "Marine Ashfall", "MS": "Marine Dense Smoke",
    "RB": "Small Craft for Rough", "RP": "Rip Currents", "SB": "Snow and Blowing", "SC": "Small Craft",
    "SE": "Hazardous Seas", "SI": "Small Craft for Winds", "SM": "Dense Smoke", "SN": "Snow",
    "SQ": "Snow Squall", "SR": "Storm", "SS": "Storm Surge", "SU": "High Surf", "SV": "Severe Thunderstorm",
    "SW": "Small Craft for Hazardous Seas", "TI": "Inland Tropical Storm", "TO": "Tornado", "TR": "Tropical Storm",
    "TS": "Tsunami", "TY": "Typhoon", "WC": "Wind Chill", "WI": "Wind", "WS": "Winter Storm",
    "WW": "Winter Weather", "ZF": "Freezing Fog", "ZR": "Freezing Rain"
}

VTEC_SIGNIFICANCE = {
    "W": "Warning", "Y": "Advisory", "A": "Watch", "S": "Statement", "O": "Outlook", "N": "Synopsis", "F": "Forecast"
}

NWS_COLORS = {
    "AF.W": "#A9A9A9", "AF.Y": "#696969", "AS.O": "#808080", "AS.Y": "#808080", "BH.S": "#40E0D0",
    "BW.Y": "#D8BFD8", "BZ.A": "#ADFF2F", "BZ.W": "#FF4500", "CF.A": "#66CDAA", "CF.S": "#6B8E23",
    "CF.W": "#228B22", "CF.Y": "#7CFC00", "DS.W": "#FFE4C4", "DS.Y": "#BDB76B", "DU.W": "#FFE4C4",
    "DU.Y": "#BDB76B", "EC.A": "#0000FF", "EC.W": "#0000FF", "EH.A": "#800000", "EH.W": "#C71585",
    "EH.Y": "#800000", "EW.W": "#FF8C00", "FA.A": "#2E8B57", "FA.W": "#00FF00", "FA.Y": "#00FF7F",
    "FF.A": "#2E8B57", "FF.S": "#8B0000", "FF.W": "#8B0000", "FG.Y": "#708090", "FL.A": "#2E8B57",
    "FL.S": "#00FF00", "FL.W": "#00FF00", "FL.Y": "#00FF7F", "FR.Y": "#6495ED", "FW.A": "#FFDEAD",
    "FW.W": "#FF1493", "FZ.A": "#00FFFF", "FZ.W": "#483D8B", "GL.A": "#FFC0CB", "GL.W": "#DDA0DD",
    "HF.A": "#9932CC", "HF.W": "#CD5C5C", "HT.Y": "#FF7F50", "HU.A": "#FF00FF", "HU.S": "#FFE4B5",
    "HU.W": "#DC143C", "HW.A": "#B8860B", "HW.W": "#DAA520", "HY.Y": "#00FF7F", "HZ.A": "#4169E1",
    "HZ.W": "#9400D3", "IS.W": "#8B008B", "LE.A": "#87CEFA", "LE.W": "#008B8B", "LE.Y": "#48D1CC",
    "LO.Y": "#A52A2A", "LS.A": "#66CDAA", "LS.S": "#6B8E23", "LS.W": "#228B22", "LS.Y": "#7CFC00",
    "LW.Y": "#D2B48C", "MA.S": "#FFDAB9", "MA.W": "#DB7093", "MF.Y": "#708090", "MH.Y": "#696969",
    "MS.Y": "#F0E68C", "RB.Y": "#D8BFD8", "RP.S": "#40E0D0", "SC.Y": "#D8BFD8", "SE.A": "#483D8B",
    "SE.W": "#D8BFD8", "SI.Y": "#D8BFD8", "SM.Y": "#F0E68C", "SQ.W": "#C71585", "SR.A": "#FFE4B5",
    "SR.W": "#9400D3", "SS.A": "#DB7FF7", "SS.W": "#C0C0C0", "SU.W": "#228B22", "SU.Y": "#BA55D3",
    "SV.A": "#DB7093", "SV.W": "#FFA500", "SW.Y": "#D8BFD8", "TO.A": "#FFFF00", "TO.W": "#FF0000",
    "TR.A": "#F08080", "TR.S": "#FFE4B5", "TR.W": "#B22222", "TS.A": "#FF00FF", "TS.W": "#FD6347",
    "TS.Y": "#D2691E", "TY.A": "#FF00FF", "TY.W": "#DC143C", "UP.A": "#4682B4", "UP.W": "#8B008B",
    "UP.Y": "#8B008B", "WC.A": "#5F9EA0", "WC.W": "#B0C4DE", "WC.Y": "#AFEEEE", "WI.Y": "#D2B48C",
    "WS.A": "#4682B4", "WS.W": "#FF69B4", "WW.Y": "#7B68EE", "ZF.Y": "#008080", "ZR.Y": "#DA70D6"
}

/tmp/ipykernel_2464842/3445465411.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Paths to the shapefiles
nws_shapefile_path = 'nws_data/wwa_202407010400_202407260400.shp'
state_shapefile_path = '/nfs/home11/ugrad/2020/tr588861/SWRCC/State_Shapefiles/state/State.shp'
county_shapefile_path = '/nfs/home11/ugrad/2020/tr588861/SWRCC/State_Shapefiles/state/Counties.shp'

# Load the NWS data
nws_data = gpd.read_file(nws_shapefile_path)
state_boundary = gpd.read_file(state_shapefile_path)
county_boundaries = gpd.read_file(county_shapefile_path)

# Load the rainfall records
rainfall_records = pd.read_csv('rainfall_records.csv')

# Function to convert UTC to Eastern Time
def convert_to_eastern(utc_time):
    # Define timezones
    eastern = timezone('US/Eastern')
    utc = timezone('UTC')

    # Localize UTC time to make it timezone-aware
    utc_time = pd.to_datetime(utc_time).tz_localize('UTC')
    
    # Convert to Eastern Time
    eastern_time = utc_time.astimezone(eastern)
    return eastern_time

# Assuming nws_data is already loaded and has an 'ISSUED' column in UTC
nws_data['ISSUED_ET'] = nws_data['ISSUED'].apply(convert_to_eastern)
nws_data['EXPIRED_ET'] = nws_data['EXPIRED'].apply(convert_to_eastern)

# Filter the data for new warnings
new_warnings = nws_data[(nws_data['STATUS'] == 'NEW') & (nws_data['GTYPE'] == 'P')]

# Convert the latitude and longitude to a list of tuples
rainfall_locations = list(zip(rainfall_records['Lat'], rainfall_records['Lon']))

# Function to create a Folium map for a specific warning type
def create_warning_map(warning_type, center=[43.0, -75.0], zoom=7):
    warning_data = new_warnings[new_warnings['PHENOM'] == warning_type]
    m = folium.Map(location=center, zoom_start=zoom)
    
    # Add state boundary to the map
    folium.GeoJson(
        state_boundary,
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 3
        }
    ).add_to(m)
    
    # Add county boundaries to the map
    folium.GeoJson(
        county_boundaries,
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'gray',
            'weight': 0.5
        }
    ).add_to(m)
    
    for _, row in warning_data.iterrows():
        # Decode the PHENOM and SIG codes
        phenom_code = row['PHENOM']
        sig_code = row['SIG']
        phenom_desc = VTEC_PHENOMENA.get(phenom_code, phenom_code)
        sig_desc = VTEC_SIGNIFICANCE.get(sig_code, sig_code)
        
        # Get the color for the warning
        color_key = f"{phenom_code}.{sig_code}"
        color = NWS_COLORS.get(color_key, '#000000')  # Default to black if not found
        
        # Extract the polygon geometry
        geom = row['geometry']
        
        # Convert Shapely geometry to GeoJSON
        geo_json = gpd.GeoSeries([geom]).__geo_interface__
        
        # Add the polygon to the map
        folium.GeoJson(
            geo_json,
            style_function=lambda x, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 2,
                'fillOpacity': 0.4
            },
            tooltip=f"{phenom_desc} {sig_desc}"
        ).add_to(m)
    
    # Add markers for rainfall records
    for _, record in rainfall_records.iterrows():
        lat = record['Lat']
        lon = record['Lon']
        folium.Marker(
            location=[lat, lon], icon=folium.Icon(color='black', icon='')
        ).add_to(m)
    
    return m

# Create maps for each warning type with markers
severe_thunderstorm_map = create_warning_map('SV')
tornado_map = create_warning_map('TO')
flash_flood_map = create_warning_map('FF')

# Save the maps in the Output_wMarkers directory
output_dir = 'Output_wMarkers'
os.makedirs(output_dir, exist_ok=True)

severe_thunderstorm_map.save(os.path.join(output_dir, 'severe_thunderstorm_map.html'))
tornado_map.save(os.path.join(output_dir, 'tornado_map.html'))
flash_flood_map.save(os.path.join(output_dir, 'flash_flood_map.html'))

# Create the combined map with markers
def create_combined_map(center=[43.0, -75.0], zoom=7):
    combined_map = folium.Map(location=center, zoom_start=zoom)
    
    # Add state boundary to the map
    folium.GeoJson(
        state_boundary,
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 3
        }
    ).add_to(combined_map)
    
    # Add county boundaries to the map
    folium.GeoJson(
        county_boundaries,
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'gray',
            'weight': 0.5
        }
    ).add_to(combined_map)
    
    for warning_type in ['SV', 'TO', 'FF']:
        warning_data = new_warnings[new_warnings['PHENOM'] == warning_type]
        
        for _, row in warning_data.iterrows():
            # Decode the PHENOM and SIG codes
            phenom_code = row['PHENOM']
            sig_code = row['SIG']
            phenom_desc = VTEC_PHENOMENA.get(phenom_code, phenom_code)
            sig_desc = VTEC_SIGNIFICANCE.get(sig_code, sig_code)
            
            # Get the color for the warning
            color_key = f"{phenom_code}.{sig_code}"
            color = NWS_COLORS.get(color_key, '#000000')  # Default to black if not found
            
            # Extract the polygon geometry
            geom = row['geometry']
            
            # Convert Shapely geometry to GeoJSON
            geo_json = gpd.GeoSeries([geom]).__geo_interface__
            
            # Add the polygon to the map
            folium.GeoJson(
                geo_json,
                style_function=lambda x, color=color: {
                    'fillColor': color,
                    'color': color,
                    'weight': 2,
                    'fillOpacity': 0.4                },
                tooltip=f"{phenom_desc} {sig_desc}"
            ).add_to(combined_map)
    
    # Add markers for rainfall records
    #for _, record in rainfall_records.iterrows():
       # lat = record['Lat']
        #lon = record['Lon']
        #folium.Marker(
           # location=[lat, lon], icon=folium.Icon(color='black', icon='')
        #).add_to(combined_map)
    
    return combined_map

# Create the combined map
combined_map = create_combined_map()

# Save the combined map in the Output_wMarkers directory
combined_map.save(os.path.join(output_dir, 'combined_warning_map.html'))

In [3]:
from selenium import webdriver
from PIL import Image
import time
import os

# Paths to your Folium HTML files
html_files = [
    'Output_wMarkers/severe_thunderstorm_map.html',
    'Output_wMarkers/tornado_map.html',
    'Output_wMarkers/flash_flood_map.html',
    'Output_wMarkers/combined_warning_map.html'
]

# Directory to save the PNG files
output_dir = 'Output_wMarkers'
os.makedirs(output_dir, exist_ok=True)

# Path to the WebDriver executable (e.g., chromedriver)
webdriver_path = 'path/to/chromedriver'

# Initialize the WebDriver (for Chrome in this case)
driver = webdriver.Chrome(executable_path=webdriver_path)

# Loop through each HTML file, render it, and save as PNG
for html_file in html_files:
    # Open the HTML file in the browser
    driver.get('file://' + html_file)

    # Allow some time for the map to fully render
    time.sleep(5)  # Adjust as needed

    # Take a screenshot of the full page
    screenshot = driver.get_screenshot_as_png()

    # Define the output PNG file path
    output_file = os.path.join(output_dir, os.path.basename(html_file).replace('.html', '.png'))

    # Save the screenshot to a file
    with open(output_file, 'wb') as f:
        f.write(screenshot)

    # Optional: Use Pillow to crop or adjust the screenshot
    with Image.open(output_file) as img:
        # Crop if necessary or adjust image as needed
        img.save(output_file, quality=95)  # Adjust quality as needed

# Close the browser
driver.quit()

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'